# Multiple-Objective Optimization Auto-Tuning

In [2]:
import paradance as hv

Load the offline dataset used for Bayesian automatic tuning.

`FILE_PATH` is the path where the training data is stored. `FILE_NAME` is the filename.

In [ ]:
loader = hv.CSVLoader(
    file_path=FILE_PATH,
    file_name=FILE_NAME,
)

For example, three different targets can be fused like this:

```
formula="targets[0] - 2 * targets[1] + 100 * max(0, targets[2])". 
```

Select the features to be used for hyperparameter search

In [ ]:
selected_columns=['pxtr_0', 'pxtr_1', 'pxtr_2', 'pxtr_3', 'pxtr_4']

We should construct a `Calculator`, to calculate the composite score based on the selected features. 

The calculation method for the composite score can be chosen as either `sum` or `product`, corresponding to the **additive** and **multiplicative** forms of the formulas.

We can also use the `free_style` type of equation, which supports any form of `eval()` expression equation.

In [ ]:
cal = hv.Calculator(
    df=loader.df, 
    selected_columns=selected_columns, 
    equation_type="product",
)

Determine the Bayesian search space: whether to search for the power term; whether to search for the first-order term of the features.

In [ ]:
power = True
first_order = False

Next, construct the optimization target based on the above settings, and determine the direction of optimization, whether to maximize or minimize the fusion target expression.

The optimization process will be recorded in the `LOG_FILE` file.

In [ ]:
ob = hv.MultipleObjective(
    direction="minimize",
    weights_num=len(selected_columns),
    formula=formula,
    power=power,
    first_order=first_order,
    log_file=LOG_FILE,
)

Add the appropriate optimizers to the optimization targets, such as the "woauc"/"auc" optimizer in this case. 

`target_column` is the sub-target that this optimizer is used to optimize, such as "watch_t"/"follow"/"click".

In [ ]:
ob.add_calculator(
    calculator=cal,
    flag='woauc',
    target_column='watch_time'
)

ob.add_calculator(
    calculator=cal,
    flag='auc',
    target_column='follow'
)

ob.add_calculator(
    calculator=cal,
    flag='auc',
    target_column='click'
)

Begin optimization, and save the tuning log. `n_trials` represents the number of rounds for hyperparameter search.

In [ ]:
ob.optimize(n_trials=600)